# Azure Machine Learning Engineer
## Project 3 - Capstone AutoML

## Create a workspace
In this step we are making an Azure Workspace and setting up an experiment

In [32]:
# Create a new workspace and define an experiment.

from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.get_details()

# Choose a name for the experiment
experiment_name = 'udacity-project-automl'
experiment = Experiment(workspace=ws, name = experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-146592
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-146592


In [33]:
# Create a compute cluster to provision VM Resources.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = 'cpu-cluster-01'

#Verify that the culster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print('Found existing cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The below information is for a locally (Azure) hosted dataset.  In this case a CSV was uploaded to the datablob store and to allow access you interogate it and go to the consume tab and copy the below code

In [23]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

from azureml.core import Workspace, Dataset

subscription_id = 'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f'
resource_group = 'aml-quickstarts-146487'
workspace_name = 'quick-starts-ws-146487'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Adult')

# create a pandas dataframe from the tabular dataset for use with sklearn functions.
df = dataset.to_pandas_dataframe()

In [34]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.dataset_factory import DataType

#file uploaded to github
url = 'https://raw.githubusercontent.com/boffyd/UdacityMLOPs-Capstone/main/adult.csv'

# pass url to Tabular dataset.  Note this is different to pandas dataframe, and gets converted to a dataframe in the function.

dataset = TabularDatasetFactory.from_delimited_files(url,header = True)
#ds = dataset.to_pandas_dataframe().dropna()

In [28]:
#ds = ds.dropna #comment out as req'd
    #df = data.dropna() #comment out as req'd needed for blobstore
     #Feature engineering look to drop correlated (i.e. education and education number features)    
    #Working class no obvious difference in categories
    #race is dominated by one race
    #native country is dominated by one country, which is in the same currency as the target
    #first filter by this then drop
    
#ds = ds[ds['native-country'].str.contains("United States")]
#ds = ds[ds['native-country'] == 'United States']
    #drop unwanted variables as mentioned above.
#df2 = df2.drop(columns=['workclass','education','race','native-country','fnlwgt','Column2'])

In [29]:
ds.head()

,Column1,age,workclass,fnlwgt,education,education-num.,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage


In [24]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data (pandas translated tabular dataset)
x, y = clean_data(df)

#automl settings have the optios to assign the dataframe and identify a target variable within it.
datafinal = pd.concat([x,y], axis = 1)

ds = dataset

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [25]:
df.head()

,Column2,age,workclass,fnlwgt,education,education-num.,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [35]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    compute_target = compute_target,
    primary_metric = 'accuracy',
    n_cross_validations = 3, #typically 5, used to make time go quicker.
    task = 'classification',
    training_data = dataset,
    label_column_name = 'wage',
    enable_onnx_compatible_models = True)

In [36]:
# Submit your automl run
experiment = Experiment(ws, 'udacity-project-automl')
automl_run = experiment.submit(config = automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster-01 with default configuration
Running on remote compute: cpu-cluster-01


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_49d2692b-7358-4369-b098-01032198c2bd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

In [26]:
pip install azureml-train-automl-runtime

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import azureml.train.automl

import azureml.automl.core
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl_run, best_automl_onnx_model = automl_run.get_output(return_onnx_model = True)
OnnxConverter.save_onnx_model(best_automl_onnx_model, file_path = 'outputs/best_automl_model.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.